<a href="https://colab.research.google.com/github/GbrlOl/whisper-fast/blob/main/Whisper_Large_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalamos las dependencias

In [1]:
!pip install --upgrade -qqq git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Cargamos el Whisper Large-V3

In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]

# result = pipe(sample)
# print(result["text"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

# Especificamos la ruta en donde se ubica el audio para transcribir

In [3]:
result = pipe("/content/CLASE_9.mp3", generate_kwargs={"language": "spanish"})
result

{'text': ' bien, entonces ahora partió esto buenas tardes no sé si está grabando sí, buenas tardes estudiantes espero que estén súper bien vamos hoy día, como les comenté en la clase pasada, hoy día la idea de la clase de hoy es ver algún problema tipo prueba, para que ustedes sepan y entiendan qué es lo que estamos trabajando me imagino que ya han estudiado recuerden que en el aula virtual ya se les dejó el problema 1 de la prueba, así que no voy a tocar nada de eso hoy día, vamos a ver otras cosas que no sean de lo que les pregunto en esa pregunta, y eso primero, principalmente eso. Así que recordar que ya vimos la materia, ya tenemos la materia completa, recordar que ya vimos la materia ya tenemos la materia completa pasamos la toda la unidad, la clase pasada terminamos de ver la parte de TN y Norton y vimos también TLEGEN, potencia, hicimos problemas y no sé si ustedes tienen dudas o consultas hay 18 alumnos ahora espero que estén súper bien bien, entonces vamos a partir, abrí si u

# Guardamos a un archivo de texto

In [23]:
with open("raw_text.txt", "w", encoding='utf-8') as arch:
  arch.write(result['text'])

In [24]:
import textwrap

with open("/content/raw_text.txt", "r") as archivo:
    lineas = archivo.readlines()

with open("transcripcion_formateada.txt", "w") as archivo_formateado:
    for linea in lineas:
        linea_formateada = "\n".join(textwrap.wrap(linea, width=160))  # Ajusta a un ancho de 80 caracteres, este puede cambiarse
        archivo_formateado.write(linea_formateada + "\n")

print("Transcripción lista...")

# Convertir a un archivo .vtt (Para marcas de tiempo)

In [28]:
def convert_to_vtt(data):
    vtt_content = "WEBVTT\n\n"

    for item in data:
        start_time = "{:02d}:{:02d}:{:02d}.{:03d}".format(int(item['timestamp'][0] // 60),
                                                           int(item['timestamp'][0] % 60),
                                                           int((item['timestamp'][0] % 1) * 1000),
                                                           int(((item['timestamp'][0] % 1) * 1000) % 1000))
        end_time = "{:02d}:{:02d}:{:02d}.{:03d}".format(int(item['timestamp'][1] // 60),
                                                         int(item['timestamp'][1] % 60),
                                                         int((item['timestamp'][1] % 1) * 1000),
                                                         int(((item['timestamp'][1] % 1) * 1000) % 1000))

        vtt_content += f"{start_time} --> {end_time}\n{item['text'].strip()}\n\n"

    return vtt_content

# data = [
#     {'timestamp': (0.0, 4.7), 'text': 'Hola a todos, Jerry aquí, y en este video estamos emocionados de presentar a Wenqi,'},
#     {'timestamp': (4.84, 9.94), 'text': 'que hablará de 12 puntos de dolor de RAG y soluciones propuestas en Lama Index.'},
#     {'timestamp': (10.48, 14.04), 'text': 'Como mucha gente sabe, que está construyendo RAG hoy en día,'},
#     {'timestamp': (14.68, 17.72), 'text': 'construir RAG de producción de performance puede ser bastante desafiante,'},
#     {'timestamp': (18.24, 22.6), 'text': 'ejemplo de último elemento sin texto completo.'}
# ]

vtt_content = convert_to_vtt(result['chunks'])

with open("transcripcion_formateada.vtt", "w") as file:
    file.write(vtt_content)

print("Archivo .vtt creado exitosamente.")


Archivo .vtt creado exitosamente.


# Descargar Audios de Youtube

In [ ]:
!pip install -qqq pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.6 MB/s eta 0:00:00


In [ ]:
from pytube import YouTube

In [ ]:
url = input('Pegar link y enter: ')

Pegar link y enter: https://www.youtube.com/watch?v=EBpT_cscTis


In [ ]:
yt_url = YouTube(url)
audio_stream = yt_url.streams.filter(only_audio=True).first()
output_path = '/content'
filename = 'audio.mp3'
audio_stream.download(output_path=output_path, filename=filename)
print(f'El audio quedó en el colab en {output_path}/{filename}')

El audio quedó en el colab en /content/audio.mp3
